In [1]:
import pandas as pd
import tqdm
import re

from konlpy.corpus import kolaw
from konlpy.tag import Okt, Hannanum, Kkma, Komoran, Twitter, Mecab
# import MeCab
from konlpy.utils import concordance, pprint

import matplotlib.pyplot as plt
import seaborn as sns

# 한글깨짐 설정
import matplotlib.font_manager as fm

# print(fm.findSystemFonts(fontpaths=None, fontext='ttf'))
# font_location = '/usr/share/fonts/truetype/NanumGothic.ttf'  #font 경로 설정
# font_name = fm.FontProperties(fname=font_location).get_name()
# plt.rc('font', family=font_name)
# plt.rc('font', size=12)        # 기본 폰트 크기

import warnings
warnings.filterwarnings('ignore')

import gensim
gensim.__version__
from gensim import corpora, models
from gensim.models import CoherenceModel
from gensim.test.utils import common_corpus
from gensim.models.wrappers import LdaMallet


In [31]:
df = pd.read_csv('./Data/novel.csv', sep='\t')
# df = pd.read_csv('./Data/novel_total_final.csv', on_bad_lines='skip')
df

,id,itemId,title,pubDate,coverSmallUrl,coverLargeUrl,publisher,priceStandard,customerReviewRank,author,isbn,link,story,review,piece
0,1,349572433,주술회전 1: 떠나가는 여름과 돌아오는 가을,20210525,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,서울미디어코믹스(서울문화사),7800,0.0,Gege Akutami(아쿠타미 제제),9791136734518,http://book.interpark.com/blog/integration/pro...,"대인기 다크 액션 판타지, 「주술회전」소설 1권 발행!!이타도리와 후시구로의 아키하...",NaN,NaN
1,2,348109712,흐르는 강물처럼(40주년 기념판),20210415,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,연암서가,15000,0.0,노먼 매클린,9791160870763,http://book.interpark.com/blog/integration/pro...,영화 〈흐르는 강물처럼〉의 원작소설영화보다 더한 감동이 굽이쳐 흐른다! 〈흐르는 강...,NaN,나는 1981년 몬태나 주를 방문하던 중에 내 친구 토머스 맥구언과 함께 미국 서부...
2,3,340692027,백작가의 비밀스런 시녀님 세트,20201027,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,필프리미엄에디션,38000,8.0,백주아,9791165655778,http://book.interpark.com/blog/integration/pro...,백주아 장편소설 『백작가의 비밀스런 시녀님 세트』. 가난한 소작농의 딸 폴라. 우연...,〈1권〉가난한 소작농의 딸 폴라.우연한 계기로 명망 높은 벨루니타 백작가의 사용인으...,NaN
3,4,343464716,신세계에서 2,20201125,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,해냄출판사,18000,9.9,기시 유스케(貴志祐介),9788965740322,http://book.interpark.com/blog/integration/pro...,일본 SF대상 수상작 & 서점대상 제6위미래의 가상세계에 빗대어 현 인류의 모순을 ...,"순백의 아이들이 평화롭게 살아가는 머나먼 미래, 그 아름다운 유토피아에 가려진 잔혹...",X. 얼굴 없는 소년. 누구보다 사랑했던 소년. 그런데 지금은 얼굴도 떠올릴 수 없...
4,5,340768143,추리소설가의 살인사건,20201028,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,소미미디어,15800,9.5,히가시노 게이고(東野圭吾),9791166111983,http://book.interpark.com/blog/integration/pro...,"추리 소설가, 편집자, 독자가 등장하는 추리소설,히가시노 게이고만이 쓸 수 있는 색...","이 작품집, 꽤나 웃기다! 한 번 읽기 시작했다면 멈출 수 없다! 올해의 페이지 터...",그것은 하마사키 회계사무소에서 온 서류였다. 소장인 하마사키 고로는 내 고등학교 동...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,1114,345966624,어니언 마말레이드 2,20210121,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,동아,12800,0.0,백서하,9791163024453,http://book.interpark.com/blog/integration/pro...,"사랑에 빠진 위그의 뒤통수를 치고,이디에트 가문의 약점을 손에 쥐게 된 비비안. “...",NaN,“상속법 관련 문제에 대해 의견을 묻고 싶네.”“죄송하지만 그 부분에 대해서는 귀족...
1114,1115,343789806,갑의 품격 1,20201130,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,가연,14500,0.0,김나든,9788968970818,http://book.interpark.com/blog/integration/pro...,"네이버 시리즈 다운로드 수 84만, 실시간 랭킹 1위.혜성처럼 나타난 김나든 작가의...","서울의 끝자락 판자촌 태생, 부모가 누군지도 모르는 고아. 보육원을 떠돌던 예일은 ...","생전 처음 보는 사람 앞에서 그렇게 예일은 펑펑 울었다. 산발이 된 머리도, 그에게..."
1115,1116,342291271,진실에 갇힌 남자,20201109,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,북로드,14800,9.5,데이비드 발다치,9791158791490,http://book.interpark.com/blog/integration/pro...,“진실이 늘 우리를 자유롭게 하는 것은 아니에요.때론 우리를 가두는 감옥이 될 수도...,"죽음을 눈앞에 두고 자신의 무죄를 주장하던 남자, 이미 죽은 것이나 다름없는 그가 ...","데커는 언젠가 땅 밑에 누워 있는 가족에게 자신도 합류하게 될 그날까지, 매년 몰리..."
1116,1117,342332559,검사 김서진 4,20201008,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,로크미디어,8000,0.0,이해날,9791135485893,http://book.interpark.com/blog/integration/pro...,No방심 작가 이해날의 검사물 리턴즈!그 검사의 목숨을 건 외줄 타기가 시작된다! ...,NaN,NaN


In [32]:
# story와 review 데이터 형태소 분리 작업

def wordSep(df):
    story = df['story']
    story = story.fillna('')

    review = df['review']
    review = review.fillna('')

    stopwords = ('하는', '된', '한', '한다', '할', '출', '해', '된다', '는', '하며', '했다', '되는', '하게', '되었다', 
                '되어', '가', '했던', '될', '하여', '그려', '낸', '되고', '하지', '하기', '와', '않는', '하면서', 
                '하다', '잘', '하고', '하는데', '쓴', '않고', '하던', '되는데', '않은', '담은', 
                '사는', '가진', '냈다', '대', '받고', '이었다', '해야', '가는', '해온', 
                '싶은', '받는', '이를', '않는다', '만나', '담아', '되면서', '담고', '다룬', '보여준다', '따라', 
                '보다',  '되지', '주는', '본', '찾아', '써', '넘어', 
                '넘나', '되', '선보이는', '준다', '해서', '일까', '만들어', '이었던', '받아', '받으며', 
                '담긴', '담겨', '갈', '하면', '나가는', '빠져', '낸다', '였다', '였던', '지', '이어', '드는' , '않았다',
                '발간', '문학', '현대문학', '베스트셀러', '권')

    necessary_type = ('Noun', 'Verb', 'Adjective')

    sep_story = []

    for s in story:
        okt = Okt()
        story_sep = okt.pos(s)
        sentence = ''
        for word,typ in story_sep:
            if (word not in stopwords) and (typ in necessary_type):
                sentence += word + ' '
        sep_story.append(sentence)

    sep_review = []

    for r in review:
        okt = Okt()
        review_sep = okt.pos(r)
        sentence = ''
        for word,typ in review_sep:
            if (word not in stopwords) and (typ in necessary_type):
                sentence += word + ' '
        sep_review.append(sentence)

    analysis_data = [s+r for s,r in zip(sep_story, sep_review)]

    return analysis_data

In [33]:
def lda_coh(stories, no_below=5, num_topics=4, data_word=None, data_words=None):
    
    # 이야기 전체 형태소 분리
    if data_word == None:
        data_words = []
        for story in stories:
            data = list(str(story).split())
            data_words.append(data)
        
    
    if data_word == 'stop':
        data_words = []
        for story in stories:
            data = list(str(story).split())
            data_words.append(data)
            
        return data_words
    
    # id2word, corpus 추출
    id2word = corpora.Dictionary(data_words)
    id2word.filter_extremes(no_below = no_below) # no_below 회 이하로 등장한 단어는 삭제
    texts = data_words
    corpus = [id2word.doc2bow(text) for text in texts]
    
    mallet_path = './mallet-2.0.8/bin/mallet' 
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, random_seed=12, id2word=id2word)

    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    
    return texts, corpus, ldamallet, coherence_ldamallet

def lda_coh_id2(stories, no_below=5, num_topics=4, data_word=None, data_words=None):
    
    # 이야기 전체 형태소 분리
    if data_word == None:
        data_words = []
        for story in stories:
            data = list(str(story).split())
            data_words.append(data)
        
    
    if data_word == 'stop':
        data_words = []
        for story in stories:
            data = list(str(story).split())
            data_words.append(data)
            
        return data_words
    
    # id2word, corpus 추출
    id2word = corpora.Dictionary(data_words)
    id2word.filter_extremes(no_below = no_below) # no_below 회 이하로 등장한 단어는 삭제
    texts = data_words
    corpus = [id2word.doc2bow(text) for text in texts]
    
    mallet_path = './mallet-2.0.8/bin/mallet' 
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, random_seed=12, id2word=id2word)

    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    
    return texts, corpus, ldamallet, coherence_ldamallet, id2word

def format_topics_sentences(texts, corpus, data_words, ldamallet, Data=df):
    # Init output

    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #ldamallet[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamallet[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamallet.show_topic(topic_num, topn=10)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    # print(type(sent_topics_df))

    # Add original text to the end of the output
    contents = pd.Series(data_words)
    # sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df = pd.concat([Data, sent_topics_df], axis=1)

    return sent_topics_df

In [34]:
def lda_make_df(df):
    analysis_data = wordSep(df)
    texts, corpus, ldamallet, coherence_ldamallet, id2word = lda_coh_id2(analysis_data, num_topics=10)
    data_words = lda_coh(analysis_data, num_topics=10, data_word='stop')
    df_topic_sents_keywords = format_topics_sentences(texts, corpus, data_words, ldamallet, Data=df)

    return df_topic_sents_keywords

In [35]:
df_lda = lda_make_df(df)

In [38]:
df_topic = df_lda[['id', 'Dominant_Topic', 'Topic_Keywords']]

In [41]:
df_topic.to_csv('novel_topic_tab.csv', header = True, index = False, encoding='utf-8-sig', sep = '\t')

In [2]:
df_topic = pd.read_csv('novel_topic_tab.csv', sep='\t')


In [10]:
df_topic.pivot_table('Dominant_Topic', 'Topic_Keywords').reset_index().sort_values('Dominant_Topic')

,Topic_Keywords,Dominant_Topic
9,"한국, 사회, 시대, 국문학, 전집, 번역, 대표, 수록, 박완서, 새로운",0
2,"마음, 시간, 서로, 두, 감정, 세상, 관계, 살, 지금, 상처",1
7,"죽음, 그녀, 아버지, 소녀, 아들, 남자, 인생, 두, 운명, 딸",2
5,"역사, 시대, 일본, 가장, 권, 전쟁, 이자, 화, 임꺽정, 중국",3
1,"내, 생각, 그녀, 눈, 더, 쪽, 몸, 있었다, 그런, 곳",4
3,"사건, 미스터리, 진실, 전, 위, 베스트셀러, 살인, 두, 일본, 스릴러",5
6,"인간, 세계, 미국, 를, 인류, 가장, 이자, 프랑스, 주인공, 세기",6
0,"권, 왕, 온, 로맨스, 드라마, 판타지, 제, 여인, 조선, 위",7
4,"아이, 청소년, 가족, 친구, 학교, 엄마, 주인공, 꿈, 문제, 소년",8
8,"집, 세계, 여성, 존재, 인물, 한국, 를, 현실, 김, 소설가",9
